<a href="https://colab.research.google.com/github/annhienktuit/UIT-AI-Challenge2020/blob/master/Vehicle_tracking_without_bells_and_whistles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sun Aug  9 04:16:57 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Clone a github project

In [2]:
!git clone https://github.com/phil-bergmann/tracking_wo_bnw

Cloning into 'tracking_wo_bnw'...
remote: Enumerating objects: 1803, done.
remote: Total 1803 (delta 0), reused 0 (delta 0), pack-reused 1803
Receiving objects: 100% (1803/1803), 6.08 MiB | 11.08 MiB/s, done.
Resolving deltas: 100% (1156/1156), done.


In [3]:
cd tracking_wo_bnw

/content/tracking_wo_bnw


#Install required dependencies

In [4]:
%%shell
pip install motmetrics==1.1.3
pip install sacred==0.8.0
pip install -U PyYAML
pip install opencv-python==4.0.1.24

     |████████████████████████████████| 143kB 4.6MB/s 
     |████████████████████████████████| 81kB 3.5MB/s 
     |████████████████████████████████| 71kB 9.3MB/s 
     |████████████████████████████████| 102kB 14.3MB/s 
     |████████████████████████████████| 163kB 54.0MB/s 
     |████████████████████████████████| 71kB 9.2MB/s 
  Created wheel for sacred: filename=sacred-0.8.0-py2.py3-none-any.whl size=94459 sha256=0ae82328540032a04e12356b0905eff4e944d6fea187479ffd534eefba7117fe
  Stored in directory: /root/.cache/pip/wheels/09/e2/7a/b85e5c1d149fce26f43dd3ecc9ceafa94665726ca07a3ab3a2
  Created wheel for jsonpickle: filename=jsonpickle-0.9.6-cp36-none-any.whl size=29466 sha256=cc95e753eceba6170079f2fbc6fb9b3fe0949f79868a9d24affe0a40342d09ae
  Stored in directory: /root/.cache/pip/wheels/07/8b/41/8ce98f4737a9ff61b1bf2673f2abfe66a6a43ad6e91d2c9736
  Created wheel for py-cpuinfo: filename=py_cpuinfo-7.0.0-cp36-none-any.whl size=20069 sha256=f138a552949d12c3391191d2a1aeaab702176b682ac1019282

In [5]:
!pip3 install -e .

Obtaining file:///content/tracking_wo_bnw
  Running setup.py develop for tracktor


#Download&Unzip MOTChallenge data

In [6]:
MOT_ROOT = '/content/'

In [7]:
!wget -nc https://motchallenge.net/data/MOT17Det.zip -P $MOT_ROOT
!wget -nc https://motchallenge.net/data/MOT16Labels.zip -P $MOT_ROOT
!wget -nc https://motchallenge.net/data/2DMOT2015.zip -P $MOT_ROOT
!wget -nc https://motchallenge.net/data/MOT16-det-dpm-raw.zip -P $MOT_ROOT
!wget -nc https://motchallenge.net/data/MOT17Labels.zip -P $MOT_ROOT

--2020-08-09 04:18:29--  https://motchallenge.net/data/MOT17Det.zip
Resolving motchallenge.net (motchallenge.net)... 131.159.19.34, 2a09:80c0:18::1034
Connecting to motchallenge.net (motchallenge.net)|131.159.19.34|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1952547113 (1.8G) [application/zip]
Saving to: ‘/content/MOT17Det.zip’

MOT17Det.zip        100%[===================>]   1.82G  19.1MB/s    in 1m 40s  

2020-08-09 04:20:10 (18.6 MB/s) - ‘/content/MOT17Det.zip’ saved [1952547113/1952547113]

--2020-08-09 04:20:10--  https://motchallenge.net/data/MOT16Labels.zip
Resolving motchallenge.net (motchallenge.net)... 131.159.19.34, 2a09:80c0:18::1034
Connecting to motchallenge.net (motchallenge.net)|131.159.19.34|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3283801 (3.1M) [application/zip]
Saving to: ‘/content/MOT16Labels.zip’

MOT16Labels.zip     100%[===================>]   3.13M  2.95MB/s    in 1.1s    

2020-08-09 04:20:12 (2.9

In [8]:
from IPython.display import clear_output

In [ ]:
# Uncomment when you want to extract MOT data to Google Drive

!unzip -d '$MOT_ROOT/MOT17Det' '$MOT_ROOT/MOT17Det.zip'
clear_output()
!unzip -d '$MOT_ROOT/MOT16Labels' '$MOT_ROOT/MOT16Labels.zip'
clear_output()
!unzip -d '$MOT_ROOT/2DMOT2015' '$MOT_ROOT/2DMOT2015.zip'
clear_output()
!unzip -d '$MOT_ROOT/MOT16-det-dpm-raw' '$MOT_ROOT/MOT16-det-dpm-raw.zip'
clear_output()
!unzip -d '$MOT_ROOT/MOT17Labels' '$MOT_ROOT/MOT17Labels.zip'
clear_output()

In [9]:
TRACKING_WITHOUT_BELLS_AND_WHISTLES = '/content/tracking_wo_bnw'

In [10]:
!unzip -d '$TRACKING_WITHOUT_BELLS_AND_WHISTLES/data/MOT17Det' '$MOT_ROOT/MOT17Det.zip'
clear_output()
!unzip -d '$TRACKING_WITHOUT_BELLS_AND_WHISTLES/data/MOT16Labels' '$MOT_ROOT/MOT16Labels.zip'
clear_output()
!unzip -d '$TRACKING_WITHOUT_BELLS_AND_WHISTLES/data/2DMOT2015' '$MOT_ROOT/2DMOT2015.zip'
clear_output()
!unzip -d '$TRACKING_WITHOUT_BELLS_AND_WHISTLES/data/MOT16-det-dpm-raw' '$MOT_ROOT/MOT16-det-dpm-raw.zip'
clear_output()
!unzip -d '$TRACKING_WITHOUT_BELLS_AND_WHISTLES/data/MOT17Labels' '$MOT_ROOT/MOT17Labels.zip'
clear_output()

#Download&Unzip object detector and re-identifiaction Siamese network weights and MOTChallenge result files

In [11]:
COLABS_ROOT = '/content/HCM-AI-Challenge 2020/colabs/Tracking without bells and whistles'

In [12]:
!wget -nc https://vision.in.tum.de/webshare/u/meinhard/tracking_wo_bnw-output_v2.zip -P '$COLABS_ROOT'

--2020-08-09 04:23:17--  https://vision.in.tum.de/webshare/u/meinhard/tracking_wo_bnw-output_v2.zip
Resolving vision.in.tum.de (vision.in.tum.de)... 131.159.19.110, 2a09:80c0:18::1110
Connecting to vision.in.tum.de (vision.in.tum.de)|131.159.19.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 273167942 (261M) [application/zip]
Saving to: ‘/content/HCM-AI-Challenge 2020/colabs/Tracking without bells and whistles/tracking_wo_bnw-output_v2.zip’

tracking_wo_bnw-out 100%[===================>] 260.51M  19.7MB/s    in 15s     

2020-08-09 04:23:32 (17.7 MB/s) - ‘/content/HCM-AI-Challenge 2020/colabs/Tracking without bells and whistles/tracking_wo_bnw-output_v2.zip’ saved [273167942/273167942]



In [13]:
!unzip -d '$TRACKING_WITHOUT_BELLS_AND_WHISTLES/' '$COLABS_ROOT/tracking_wo_bnw-output_v2.zip'


Archive:  /content/HCM-AI-Challenge 2020/colabs/Tracking without bells and whistles/tracking_wo_bnw-output_v2.zip
   creating: /content/tracking_wo_bnw/output/faster_rcnn_fpn_training_mot_17/
  inflating: /content/tracking_wo_bnw/output/faster_rcnn_fpn_training_mot_17/.DS_Store  
   creating: /content/tracking_wo_bnw/__MACOSX/
   creating: /content/tracking_wo_bnw/__MACOSX/output/
   creating: /content/tracking_wo_bnw/__MACOSX/output/faster_rcnn_fpn_training_mot_17/
  inflating: /content/tracking_wo_bnw/__MACOSX/output/faster_rcnn_fpn_training_mot_17/._.DS_Store  
  inflating: /content/tracking_wo_bnw/output/faster_rcnn_fpn_training_mot_17/MOT17-04.txt  
  inflating: /content/tracking_wo_bnw/__MACOSX/output/faster_rcnn_fpn_training_mot_17/._MOT17-04.txt  
  inflating: /content/tracking_wo_bnw/output/faster_rcnn_fpn_training_mot_17/MOT17-10.txt  
  inflating: /content/tracking_wo_bnw/__MACOSX/output/faster_rcnn_fpn_training_mot_17/._MOT17-10.txt  
  inflating: /content/tracking_wo_bnw/o

#Evaluate Tracktor

In [14]:
!pip install TrackTor
!pip install motmetrics

In [15]:
!python experiments/scripts/test_tracktor.py

WARNING - test_tracktor - No observers have been added to this run
INFO - test_tracktor - Running command 'main'
INFO - test_tracktor - Started
Configuration (modified, added, typechanged, doc):
  seed = 453695
  reid:
    db_train = 'train'
    db_val = False
    desription = 'all'
    lr_scheduler = True
    max_epochs = 69
    module_name = 'reid'
    name = 'res50-mot17-batch_hard'
    seed = 12345
    cnn:
      output_dim = 128
    dataloader:
      K = 4
      P = 18
      crop_H = 256
      crop_W = 128
      max_per_person = 1000
      normalize_mean = [0.485, 0.456, 0.406]
      normalize_std = [0.229, 0.224, 0.225]
      transform = 'random'
      vis_threshold = 0.3
    model_args:
      loss = 'batch_hard'
      margin = 0.2
      prec_at_k = 3
    solver:
      optim = 'Adam'
      optim_args:
        lr = 0.0003
        weight_decay = 0.0
  tracktor:
    dataset = 'mot17_train_FRCNN17'
    desription = None
    frame_split = [0.0, 1.0]
    interpolate = False
    module_

#Run on Custom Video

##Generate seqinfo.ini for running on custom video

In [16]:
mkdir -p src/preprocess_data/

In [17]:
import configparser
import os
import cv2
import glob
import sys
import subprocess


# Due to some issues of cap.get(cv2.CAP_PROP_FRAME_COUNT) when running on several videos, 
# we build our own function to calculate the total frames of a video.
def get_total_frame(video_path, save_path, is_save=True):
    cap = cv2.VideoCapture(video_path)
    cnt = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if ret is False:
            break
        if is_save:
            cv2.imwrite(
                os.path.join(save_path, str(cnt+1).zfill(6) + '.jpg'), frame)
        cnt += 1
    return cnt


def get_duration(cap):
    # duration = total_frame / FPS
    return cap.get(cv2.CAP_PROP_FRAME_COUNT) / cap.get(cv2.CAP_PROP_FPS)


def build_video_config_ini(video_path, save_path):
    print(f'Processing video: {video_path}')
    video_name = os.path.splitext(os.path.basename(video_path))[0]

    video_save_root = os.path.join(save_path, video_name)
    os.makedirs(video_save_root, exist_ok=True)

    img_dir = 'img1'
    frames_save_root = os.path.join(video_save_root, img_dir)
    os.makedirs(frames_save_root, exist_ok=True)

    cap = cv2.VideoCapture(video_path)

    num_frames = get_total_frame(
        video_path, save_path=frames_save_root, is_save=True)
    duration = get_duration(cap)
    fps = num_frames/duration
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    im_ext = '.jpg'

    config = configparser.ConfigParser()
    config.add_section('Sequence')
    config.set('Sequence', 'name', video_name)
    config.set('Sequence', 'imDir', img_dir)
    config.set('Sequence', 'frameRate', f'{round(fps)}')
    config.set('Sequence', 'seqLength', f'{num_frames}')
    config.set('Sequence', 'imWidth', f'{round(width)}')
    config.set('Sequence', 'imHeight', f'{round(height)}')
    config.set('Sequence', 'imExt', im_ext)

    print(f'Num Frames: {num_frames}')
    print(f'Duration: {duration}')
    print(f'FPS: {fps}')
    print(f'Height: {height}')
    print(f'Width: {width}')

    cfg_file = os.path.join(video_save_root, 'seqinfo.ini')
    with open(cfg_file, 'w') as f:
        config.write(f)

    cap.release()

###Run the following command, then **upload your video to given path**!

In [18]:
mkdir -p /content/tracking_wo_bnw/data/my_video

In [19]:
build_video_config_ini(video_path='/content/tracking_wo_bnw/data/my_video/nkkn.mp4',
                       save_path='/content/tracking_wo_bnw/data/my_video')

Processing video: /content/tracking_wo_bnw/data/my_video/nkkn.mp4
Num Frames: 145
Duration: 5.8
FPS: 25.0
Height: 450.0
Width: 800.0


## Run Object Detection on custom video

In [20]:
mkdir -p /content/tracking_wo_bnw/output/faster_rcnn_fpn_training_coco/

In [21]:
import torchvision
import torch

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor


model = torchvision.models.detection.fasterrcnn_resnet50_fpn(
    pretrained=True) # get FasterRCNN model pretrained on COCO with 91 classes
torch.save(model.state_dict(),
            '/content/tracking_wo_bnw/output/faster_rcnn_fpn_training_coco/frcnn.model')

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


In [22]:
mkdir -p /content/tracking_wo_bnw/data/my_video_label

In [23]:
import torch
import cv2
import os
import glob
import sys
import numpy as np

from torchvision import transforms as T
from torch.utils.data import Dataset, DataLoader
sys.path.append('/content/tracking_wo_bnw/src/tracktor')
from frcnn_fpn import FRCNN_FPN
from PIL import Image
from natsort import natsorted


def detect(frames_info_dir, save_path):
    # Load FRCNN_FPN pretrained model
    obj_detect = FRCNN_FPN(num_classes=91)
    obj_detect.load_state_dict(torch.load(
        '/content/tracking_wo_bnw/output/faster_rcnn_fpn_training_coco/frcnn.model'))
    obj_detect.eval()

    dirname = os.path.basename(frames_info_dir)
    frames_root = os.path.join(frames_info_dir, 'img1')

    save_path = os.path.join(save_path, dirname)
    os.makedirs(save_path, exist_ok=True)
    det_dir = os.path.join(save_path, 'det')
    os.makedirs(det_dir, exist_ok=True)

    det_txt = os.path.join(det_dir, 'det.txt')
    
    with open(det_txt, 'w') as f:

      img_list = []
      for frame_path in natsorted(glob.glob(os.path.join(frames_root, '*.jpg'))):
          frame_name, _ = os.path.splitext(os.path.basename(frame_path))
          frame_id = int(frame_name)  # adding one temporarily

          img = Image.open(frame_path).convert('RGB')
          img = T.ToTensor()(img)
          img = img.unsqueeze(1)
          vis_img = cv2.imread(frame_path)

          bboxes, scores = obj_detect.detect(img)

          for bbox, score in zip(bboxes, scores):
              x1, y1, x2, y2 = bbox
              width = x2 - x1 + 1
              height = y2 - y1 + 1

              line = ','.join((str(frame_id), '-1', str(x1.item()), str(y1.item()),
                                str(width.item()), str(height.item()), str(score.item())))
              print(line)
              f.write(line + '\n')

          # For visualization purpose
          # for bbox in bboxes:
          # 	x1, y1, x2, y2 = [int(c) for c in bbox]

          # 	cv2.rectangle(vis_img, (x1, y1), (x2, y2), (0, 255, 0), 2)

          # cv2.imshow('fig', vis_img)
          # cv2.waitKey(0)
          # cv2.destroyAllWindows()

In [24]:
detect(frames_info_dir='/content/tracking_wo_bnw/data/my_video/nkkn',
       save_path='/content/tracking_wo_bnw/data/my_video_label/output_nkkn')

/usr/local/lib/python3.6/dist-packages/torchvision/ops/boxes.py:101: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  keep = keep.nonzero().squeeze(1)


Streaming output truncated to the last 5000 lines.
96,-1,231.49351501464844,354.2107849121094,48.42802429199219,82.33175659179688,0.9936283230781555
96,-1,175.71730041503906,270.1088562011719,39.27748107910156,68.08053588867188,0.9930304884910583
96,-1,164.8325958251953,213.29457092285156,25.32843017578125,51.63563537597656,0.9921813011169434
96,-1,525.1195068359375,261.9609680175781,163.59063720703125,137.17706298828125,0.9918270111083984
96,-1,320.59014892578125,280.2676086425781,34.880950927734375,66.8663330078125,0.9875026345252991
96,-1,426.25518798828125,155.94491577148438,49.17535400390625,39.265380859375,0.9799322485923767
96,-1,458.10302734375,291.82037353515625,38.383148193359375,60.028045654296875,0.9797168374061584
96,-1,176.27992248535156,300.70709228515625,34.99305725097656,59.930755615234375,0.9790889620780945
96,-1,589.9131469726562,240.72848510742188,151.14947509765625,101.28353881835938,0.9757359623908997
96,-1,320.57855224609375,302.8611145019531,35.384368896484375,6

##Run track

### Copy video frames and detection info to specific path, then rename the directory to the appropriate format as MOT Challenge data

In [27]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [29]:
cp -R "/content/tracking_wo_bnw/data/my_video_label/output_nkkn/nkkn/det/det.txt" "/content/drive/My Drive/Backup UIT"

In [26]:
%%shell

# cp folder contain Video frames (being extracted from given custom video as above)
cp -r "/content/tracking_wo_bnw/data/my_video/nkkn" "/content/tracking_wo_bnw/data/MOT17Det/test"
# rename folder to MOT data format, here we rename to MOT17-15. You can set filename to MOT17-xx with xx: MOT17-16, MOT17-17,...
mv "/content/tracking_wo_bnw/data/MOT17Det/test/nkkn" "/content/tracking_wo_bnw/data/MOT17Det/test/MOT17-15"

# similar for Detection info folder
cp -r "/content/tracking_wo_bnw/data/my_video_label/nkkn/content/tracking_wo_bnw/data/MOT17Labels/test"
# here the folder is renamed to `MOT17-15-FRCNN` (FRCNN is the method used for object detection)
mv "/content/tracking_wo_bnw/data/MOT17Labels/test/nkkn" "/content/tracking_wo_bnw/data/MOT17Labels/test/MOT17-15-FRCNN"

cp: missing destination file operand after '/content/tracking_wo_bnw/data/my_video_label/nkkn/content/tracking_wo_bnw/data/MOT17Labels/test'
Try 'cp --help' for more information.
mv: cannot stat '/content/tracking_wo_bnw/data/MOT17Labels/test/nkkn': No such file or directory


CalledProcessError: ignored

### To run tracking on specific video data, you need to update `test_sequences` variable in /content/tracking_wo_bnw/src/tracktor/datasets/mot_wrapper.py

For example, change test_sequences to ['MOT17-15'] if you want to run only on  the newly created video data

###To track specific classes (e.g.: car, truck, motorcycles,...), you need to update /content/tracking_wo_bnw/src/tracktor/frcnn_fpn.py

Specifically, you need to comment out lines 61-63, and replace with the following lines: 

        pred_boxes = pred_boxes[:, 3:4].squeeze(dim=1).detach()  # for car
        pred_boxes = resize_boxes(pred_boxes, self.preprocessed_images.image_sizes[0], self.original_image_sizes[0])
        pred_scores = pred_scores[:, 3:4].squeeze(dim=1).detach()

### In /content/tracking_wo_bnw/experiments/scripts/test_tracktor.py, change:


```
obj_detect = FRCNN_FPN(num_classes=2)
```
to:
```
obj_detect = FRCNN_FPN(num_classes=91)
```

###To run Tracktor on custom video file using FRCNN-FPN pretrained on COCO, you need to update some lines in the following file: /content/tracking_wo_bnw/experiments/cfgs/tracktor.yaml

* path to the pretrained model
```
obj_detect_model: /content/tracking_wo_bnw/output/faster_rcnn_fpn_training_coco/frcnn.model
```

* dataset name
```
dataset: mot17_test_FRCNN17
```

* write visualized images to disk
```
write_images: True
```

###You can also try experimenting with different params.

In [30]:
!python experiments/scripts/test_tracktor.py

WARNING - test_tracktor - No observers have been added to this run
INFO - test_tracktor - Running command 'main'
INFO - test_tracktor - Started
Configuration (modified, added, typechanged, doc):
  seed = 453695
  reid:
    db_train = 'train'
    db_val = False
    desription = 'all'
    lr_scheduler = True
    max_epochs = 69
    module_name = 'reid'
    name = 'res50-mot17-batch_hard'
    seed = 12345
    cnn:
      output_dim = 128
    dataloader:
      K = 4
      P = 18
      crop_H = 256
      crop_W = 128
      max_per_person = 1000
      normalize_mean = [0.485, 0.456, 0.406]
      normalize_std = [0.229, 0.224, 0.225]
      transform = 'random'
      vis_threshold = 0.3
    model_args:
      loss = 'batch_hard'
      margin = 0.2
      prec_at_k = 3
    solver:
      optim = 'Adam'
      optim_args:
        lr = 0.0003
        weight_decay = 0.0
  tracktor:
    dataset = 'mot17_test_FRCNN17'
    desription = None
    frame_split = [0.0, 1.0]
    interpolate = False
    module_n

## Visualize Tracking Video

In [31]:
cd /content/tracking_wo_bnw/output/tracktor/MOT17/Tracktor++/mot17_test_FRCNN17/MOT17-15-FRCNN

/content/tracking_wo_bnw/output/tracktor/MOT17/Tracktor++/mot17_test_FRCNN17/MOT17-15-FRCNN


###Concatenate frames

In [32]:
!ffmpeg -f image2 -framerate 15 -i %06d.jpg -vcodec libx264 -y movie.mp4 -vf scale=320:-1

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

# New Section

In [33]:
!pip install -U kora
from kora.drive import upload_public
url = upload_public('movie.mp4')
# then display it
from IPython.display import HTML
HTML(f"""<video src={url} width=1000 controls/>""")

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section